In [ ]:

from shapely.geometry import MultiPolygon
from geospatial_optimization import (
    optimize_sensor_placement,
    calculate_placement_metrics,
    plot_sensor_map,
    export_to_geojson,
)

# Define the operational area boundary
operational_area = MultiPolygon([
    [
        [
            (-15.63607, 63.44591),
            (-14.87971, 64.01143),
            (-13.44727, 64.15853),
            (-9.47021, 63.56812),
            (-7.85655, 62.73101),
            (-8.02002, 61.89758),
            (-9.42634, 61.81594),
            (-13.02268, 62.61201),
        ],
        [],
    ],
    [
        [
            (-5.9332, 61.79661),
            (-5.34871, 62.16978),
            (-3.53776, 61.98027),
            (-1.604, 61.27023),
            (-1.4502, 60.91975),
            (-2.41515, 60.37148),
            (-3.83423, 60.58531),
            (-5.625, 61.12202),
            (-6.1084, 61.53317),
        ],
        [],
    ],
    [
        [
            (-26.06663, 68.64959),
            (-24.60937, 68.08971),
            (-22.58789, 66.65298),
            (-24.52148, 66.00015),
            (-26.5078, 66.31433),
            (-28.82553, 67.85732),
        ],
        [],
    ],
])


In [ ]:

configurations = []
azimuths = [i * 22.5 for i in range(16)]
for a in azimuths:
    configurations.append({"azimuth_degree": a, "fan_degree": 50, "range_km": 130})
    configurations.append({"azimuth_degree": a, "fan_degree": 90, "range_km": 100})
    configurations.append({"azimuth_degree": a, "fan_degree": 130, "range_km": 70})


In [ ]:

placed_sensors_info, status = optimize_sensor_placement(
    operational_area,
    configurations,
    resolution_km=30,
    max_sensors=99,
    coverage_requirement=0.80,
    encourage_overlapping=0.5,
)
placed_sensors_polygons, metrics = calculate_placement_metrics(
    operational_area,
    placed_sensors_info,
    resolution_km=30,
)
print(f'Status: {status}')
print(f'Number of sensors placed: {len(placed_sensors_info)}')
print(f"Estimated coverage percentage: {metrics['estimated_coverage']*100:.2f}%")
print(f"Actual Area Coverage: {metrics['area_coverage']*100:.2f}%")


In [ ]:

plot_sensor_map(operational_area, placed_sensors_polygons, placed_sensors_info)


In [ ]:

export_to_geojson('sensor_placement.geojson', operational_area, placed_sensors_info)
